In [3]:
!pip install sentencepiece

In [4]:
!pip install soynlp

In [16]:
import pandas as pd

train = pd.read_csv('./train.csv')
dev = pd.read_csv('dev.csv')
test = pd.read_csv('test.csv')

In [7]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# T5 모델 로드
model = T5ForConditionalGeneration.from_pretrained("j5ng/et5-typos-corrector")
tokenizer = T5Tokenizer.from_pretrained("j5ng/et5-typos-corrector")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = model.to(device)

In [ ]:
def correct_spelling(input_text):
    # 입력 문장 인코딩
    input_encoding = tokenizer("맞춤법을 고쳐주세요: " + input_text, return_tensors="pt")
    input_ids = input_encoding.input_ids.to(device)
    attention_mask = input_encoding.attention_mask.to(device)

    # T5 모델 출력 생성
    output_encoding = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=128,
        num_beams=5,
        early_stopping=True,
    )

    # 출력 문장 디코딩
    output_text = tokenizer.decode(output_encoding[0], skip_special_tokens=True)

    return output_text

In [1]:
def apply_correct(data):
    data['sentence_1'] = data['sentence_1'].apply(correct_spelling)
    data['sentence_2'] = data['sentence_2'].apply(correct_spelling)
    return data

In [ ]:
train = apply_correct(train)
test = apply_correct(train)
dev = apply_correct(dev)

In [22]:
train.to_csv('correct_train.csv', index = False)
dev.to_csv('correct_dev.csv', index = False)
test.to_csv('correct_test.csv', index = False)

In [21]:
train = pd.read_csv('correct_train.csv')
dev = pd.read_csv('correct_dev.csv')
test = pd.read_csv('correct_test.csv')

In [22]:
train

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도 있고 반전도 있고 웬만한 한국 영화들하고는 차원이 다르네요~.,반전도 있고 사랑도 있고 재미도 있네요.,2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권이 없다고 뜹니다 ;;.,오 액세스권이 없다고 합니다.,4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택 청약 조건 변경해 주세요.,주택청약 무주택 기준 변경해 주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사 후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯하네요!.,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!.,0.0,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,교원 능력 개발 평가에서 교원이 보호받을 수 있는 장치를 마련해야 합니다.,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요.,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성부의 폐지를 원합니다.,여성가족부 폐지를 청원합니다.,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,국회의원 월급 좀 줄여주세요.,공무원 봉급 좀 줄이지 좀 마세요.,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못 한 점심은 다음에 다시 츄라이 하기로 해요!.,오늘 못 먹은 밥은 꼭 담에 먹기로 하고요!.,3.2,1.0


In [23]:
test

,id,source,sentence_1,sentence_2
0,boostcamp-sts-v1-test-000,petition-sampled,가상 화폐 거래소 폐쇄하지 말고.,가상 화폐 거래소 폐쇄 지지합니다.
1,boostcamp-sts-v1-test-001,petition-sampled,뇌물적폐 1호 640만 달라 70억 뇌물 받은 권양숙 구속하고 재산이 몰수하라.,뇌물적폐 원조 640만 달라 70억 뇌물 받은 권양숙 구속하고 재산이 몰수하세요.
2,boostcamp-sts-v1-test-002,petition-rtt,기무사 영관급의 하극상 정말 이대로 방관하는 게 민주주의인지요.,그냥 가만히 있는 게 진짜 민주주의인가요?.
3,boostcamp-sts-v1-test-003,nsmc-sampled,화까지가 한계였다.,기대가 너무 컸다.
4,boostcamp-sts-v1-test-004,slack-rtt,왜 혼자 있지? ᅲᅲ.,왜 혼자야 ᅲᅲ.
...,...,...,...,...
1095,boostcamp-sts-v1-test-1095,slack-sampled,슬랙 채널 개편: 현재 75개 채널들이 있는데 사용되지 않는 채널들은 정리하고 넘버...,오늘 슬랙의 채널 및 사용권한에 대한 변경이 있을 예정입니다!.
1096,boostcamp-sts-v1-test-1096,petition-rtt,청소년 보호법 폐지 청원합니다.,청소년 보호법 폐지 청원서.
1097,boostcamp-sts-v1-test-1097,petition-rtt,존경하는 대통령님 상서 경유 자동차 단속 대상은 매연이 많이 나오는 화물 자동차에 ...,친애하는 문 대통령님! 디젤 차량 단속이 매연을 내는 트럭으로 제한되기를 간절히 기...
1098,boostcamp-sts-v1-test-1098,nsmc-rtt,요즘 재미가 훅 떨어짐.,요즘 재미가 사라졌다.


In [24]:
dev

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-dev-000,nsmc-sampled,액션은 개뿔 총 몇 번 쏘고 끝입니다.,액션은 흉내만 내고 그마저도 후반부에는 슬로우 모션 처리.,2.0,0.0
1,boostcamp-sts-v1-dev-001,slack-rtt,감격스러워 입 막으심?.,너무 감동해서 입 다물어?.,3.4,1.0
2,boostcamp-sts-v1-dev-002,nsmc-rtt,이번 연도에 본 영화 중 가장 최악의 영화.,올해 본 영화 중 최악.,4.0,1.0
3,boostcamp-sts-v1-dev-003,slack-rtt,특히 평소 뮤직 채널에 많은 영감을 불어넣어 주시는!.,특히! 당신은 항상 많은 음악 채널에 영감을 줍니다!.,3.4,1.0
4,boostcamp-sts-v1-dev-004,slack-sampled,다음 밥스테이지가 기대됩니다~ ᄒ.,다음 후기도 기대됩니다~~.,1.4,0.0
...,...,...,...,...,...,...
545,boostcamp-sts-v1-dev-545,nsmc-sampled,"사회적 이슈를 다루고 있는 가슴 찡한 드라마네요,,,.",정말 가슴을 따뜻하게 한 좋은 드라마.,2.0,0.0
546,boostcamp-sts-v1-dev-546,slack-rtt,비타민 먹는 장면.,비타민을 먹는 장면.,4.8,1.0
547,boostcamp-sts-v1-dev-547,nsmc-sampled,내용이 뭔 내용인지도 모르겠음.,무슨 의미로 만들었는지 모르겠음.,2.4,0.0
548,boostcamp-sts-v1-dev-548,slack-rtt,예: 주말에는 개인 캘린더만 업무 시간에는 업무 캘린더만 보기.,예: 주말에는 개인 캘린더만 보고 업무 시간에는 업무 캘린더만 보기.,5.0,1.0


- correct_train : 오타교정기만 적용

### 문장부호 제거 및 반복되는 문자 제거

In [25]:
import re
from soynlp.normalizer import *

def last_preprocess(text):
    # 한글, 공백, <PERSON>을 제외한 모든 문자와 문장부호 제거
    result = re.sub(r'[^ㄱ-ㅣ가-힣a-zA-Z0-9\s<PERSON>]', '', text)
    result = repeat_normalize(result, num_repeats=2) ## ㅋㅋㅋㅋㅋ, ㅎㅎㅎㅎ 같은 반복 제거
    return result

# 테스트
text = '<English> 문장! 테스트.....~ㅋㅋㅋㅋㅋ뿌뿌!^^NNNN9696'
result = last_preprocess(text)
print(result)


<English> 문장 테스트ㅋㅋ뿌뿌NN9696


In [26]:
def apply_last(data):
    data['sentence_1'] = data['sentence_1'].apply(last_preprocess)
    data['sentence_2'] = data['sentence_2'].apply(last_preprocess)
    return data

In [27]:
train = apply_last(train)
dev = apply_last(dev)
test = apply_last(test)

In [28]:
train.head()

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도 있고 반전도 있고 웬만한 한국 영화들하고는 차원이 다르네요,반전도 있고 사랑도 있고 재미도 있네요,2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권이 없다고 뜹니다,오 액세스권이 없다고 합니다,4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택 청약 조건 변경해 주세요,주택청약 무주택 기준 변경해 주세요,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사 후 처음 대면으로 만나 반가웠습니다,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯하네요,꼬옥 실제로 한번 뵈어요 뿌뿌뿌,0.0,0.0


In [29]:
dev.head()

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-dev-000,nsmc-sampled,액션은 개뿔 총 몇 번 쏘고 끝입니다,액션은 흉내만 내고 그마저도 후반부에는 슬로우 모션 처리,2.0,0.0
1,boostcamp-sts-v1-dev-001,slack-rtt,감격스러워 입 막으심,너무 감동해서 입 다물어,3.4,1.0
2,boostcamp-sts-v1-dev-002,nsmc-rtt,이번 연도에 본 영화 중 가장 최악의 영화,올해 본 영화 중 최악,4.0,1.0
3,boostcamp-sts-v1-dev-003,slack-rtt,특히 평소 뮤직 채널에 많은 영감을 불어넣어 주시는,특히 당신은 항상 많은 음악 채널에 영감을 줍니다,3.4,1.0
4,boostcamp-sts-v1-dev-004,slack-sampled,다음 밥스테이지가 기대됩니다,다음 후기도 기대됩니다,1.4,0.0


In [30]:
test.head()

,id,source,sentence_1,sentence_2
0,boostcamp-sts-v1-test-000,petition-sampled,가상 화폐 거래소 폐쇄하지 말고,가상 화폐 거래소 폐쇄 지지합니다
1,boostcamp-sts-v1-test-001,petition-sampled,뇌물적폐 1호 640만 달라 70억 뇌물 받은 권양숙 구속하고 재산이 몰수하라,뇌물적폐 원조 640만 달라 70억 뇌물 받은 권양숙 구속하고 재산이 몰수하세요
2,boostcamp-sts-v1-test-002,petition-rtt,기무사 영관급의 하극상 정말 이대로 방관하는 게 민주주의인지요,그냥 가만히 있는 게 진짜 민주주의인가요
3,boostcamp-sts-v1-test-003,nsmc-sampled,화까지가 한계였다,기대가 너무 컸다
4,boostcamp-sts-v1-test-004,slack-rtt,왜 혼자 있지,왜 혼자야


In [63]:
train.to_csv('train_correct_pre.csv',index = False)
dev.to_csv('dev_correct_pre.csv',index = False)
test.to_csv('test_correct_pre.csv',index = False)

In [44]:
# 결측치 발생 -> 결측치 처리
a = pd.read_csv('train_correct_pre.csv')

In [73]:
a = a.dropna()

In [75]:
a.to_csv('train_correct_pre.csv', index = False)